In [2]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from keras.datasets import cifar10


print(f"Tensorflow {tf.__version__}")

2025-10-24 14:57:20.804174: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-24 14:57:20.805715: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-24 14:57:20.920256: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-24 14:57:22.977749: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

Tensorflow 2.20.0


https://www.kaggle.com/code/kedarsai/cifar-10-88-accuracy-using-keras#Model-5:-Adding-Batch-Normalization

In [10]:

def resnet_v1_eembc():

    inputs = tf.keras.Input(shape = (32, 32, 3))
    x = tf.keras.layers.Conv2D(16, kernel_size=(5,5),activation='relu',padding = 'same',use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(64,kernel_size=(5,5),activation='relu',padding = 'same',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(64,kernel_size=(5,5),activation='relu',padding = 'same',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(10,activation='softmax',use_bias=False)(x)

    # Create functional model
    model= Model(inputs, x)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

    # Loss function
    model.compile(
        loss='categorical_crossentropy', # loss function
        optimizer=optimizer, # learning rule
        metrics=['accuracy'] # show accuracy
    )
    print(model.summary())

    return model

In [ ]:
# Download data
(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()

# Normalize inputs
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.
  
# One-hot output vectors
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, 10)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, 10)

In [12]:
model = resnet_v1_eembc()

# Train model (use one-hot labels)
history = model.fit(
    train_data, train_labels_onehot,       # training data (one-hot labels)
    batch_size=128,                        # batch size
    epochs=20,                              # Maximum number of epochs
    validation_split=0.1,                  # Percentage of training data used for validation
)

# Test model: get class predictions and evaluate with one-hot labels
predictions_keras = np.argmax(model.predict(test_data, verbose=0), axis=1)
test_loss, test_accuracy = model.evaluate(test_data, test_labels_onehot, verbose=0)
print(f"Test accuracy: {test_accuracy}")
model.save("CIFAR_v2.keras")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 16)     │         1,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        25,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 64)       │       102,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        10,240 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 140,016 (546.94 KB)

 Trainable params: 139,728 (545.81 KB)

 Non-trainable params: 288 (1.12 KB)

None
Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 44s 123ms/step - accuracy: 0.3910 - loss: 1.8463 - val_accuracy: 0.4636 - val_loss: 1.5593
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 39s 112ms/step - accuracy: 0.5047 - loss: 1.4270 - val_accuracy: 0.4758 - val_loss: 1.4366
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 41s 117ms/step - accuracy: 0.5607 - loss: 1.2595 - val_accuracy: 0.5650 - val_loss: 1.2719
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 40s 114ms/step - accuracy: 0.6028 - loss: 1.1400 - val_accuracy: 0.4022 - val_loss: 1.9277
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 39s 111ms/step - accuracy: 0.6265 - loss: 1.0662 - val_accuracy: 0.5726 - val_loss: 1.2495
Epoch 6/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 41s 118ms/step - accuracy: 0.6491 - loss: 1.0007 - val_accuracy: 0.3324 - val_loss: 2.5460
Epoch 7/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 39s 112ms/step - accuracy: 0.6688 - loss: 0.9485 - val_accuracy: 0.6228 - val_loss: 1.1220
Epoch 8/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 41s 116ms/step - accuracy: 0.6843 - lo

In [ ]:
import ANNarchy
from ANNarchy.extensions.ann_to_snn_conversion import ANNtoSNNConverter
ANNarchy.clear()
snn_converter = ANNtoSNNConverter(
    input_encoding='IB', 
    hidden_neuron='IaF',
    read_out='spike_count',
)


ANNarchy 4.8 (4.8.2.5) on linux (posix).


In [ ]:
net = snn_converter.load_keras_model(
    "CIFAR_v2.keras", 
    show_info=True)

2025-10-24 14:57:36.208692: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


* Input layer: input_layer, (32, 32, 3)
* InputLayer skipped.
* Conv2D layer: conv2d, (32, 32, 16) 
* BatchNormalization skipped.
* MaxPooling2D layer: max_pooling2d, (16, 16, 16) 
* Conv2D layer: conv2d_1, (16, 16, 64) 
* BatchNormalization skipped.
* MaxPooling2D layer: max_pooling2d_1, (8, 8, 64) 
* Conv2D layer: conv2d_2, (8, 8, 64) 
* BatchNormalization skipped.
* MaxPooling2D layer: max_pooling2d_2, (4, 4, 64) 
* Dropout skipped.
* Flatten skipped.
* Dense layer: dense, 10 
    weights: (10, 1024)
    mean -0.0005523908184841275, std 0.038356706500053406
    min -0.14490018784999847, max 0.1360320895910263



In [9]:
predictions_snn = snn_converter.predict(test_data[:100], duration_per_sample=200)

100%|██████████| 100/100 [07:20<00:00,  4.41s/it]


In [10]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

print(classification_report(test_labels[:100], predictions_snn))
print("Test accuracy of the SNN:", accuracy_score(test_labels[:100], predictions_snn))

              precision    recall  f1-score   support

           0       0.25      0.20      0.22        10
           1       0.00      0.00      0.00         6
           2       0.33      0.12      0.18         8
           3       0.50      0.10      0.17        10
           4       0.50      0.14      0.22         7
           5       0.28      0.88      0.42         8
           6       0.20      0.06      0.10        16
           7       0.13      0.55      0.21        11
           8       0.00      0.00      0.00        13
           9       0.00      0.00      0.00        11

    accuracy                           0.19       100
   macro avg       0.22      0.21      0.15       100
weighted avg       0.21      0.19      0.14       100

Test accuracy of the SNN: 0.19


In [11]:
model = tf.keras.models.load_model('CIFAR_v2.keras')
loss, accuracy = model.evaluate(test_data, test_labels_onehot, verbose=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.6630 - loss: 1.0635


In [13]:
print(f"Test accuracy of the ANN: {accuracy:.4f}")
print("Test accuracy of the SNN:", accuracy_score(test_labels[:100], predictions_snn))

Test accuracy of the ANN: 0.6630
Test accuracy of the SNN: 0.19
